In [ ]:
import os
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
import librosa.display
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
try:
    from spela.spectrogram import Spectrogram 
    from spela.melspectrogram import Melspectrogram
except:
    !pip install spela
    from spela.spectrogram import Spectrogram 
    from spela.melspectrogram import Melspectrogram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
data_dir = "/content/drive/MyDrive/archive (3)/16000_pcm_speeches"

In [ ]:
os.listdir(data_dir)

['tf_Wav_reader.py',
 'Julia_Gillard',
 'Nelson_Mandela',
 'other',
 'Jens_Stoltenberg',
 'Magaret_Tarcher',
 'Benjamin_Netanyau',
 '_background_noise_']

In [ ]:
nelson_madela = [item for item in os.listdir(data_dir + "/Nelson_Mandela")]
nelson_madela[:10]

['1439.wav',
 '1472.wav',
 '1474.wav',
 '1475.wav',
 '1457.wav',
 '1458.wav',
 '1468.wav',
 '1476.wav',
 '1453.wav',
 '1459.wav']

In [ ]:
def compute_spectrogram_melspectrogram_and_plot(wav_dir, compute_type):
    with tf.compat.v1.Session(graph=tf.compat.v1.Graph()) as sess:
        wav_filename_placeholder = tf.compat.v1.placeholder(tf.compat.v1.string, [])
        wav_loader = tf.io.read_file(wav_filename_placeholder)
        wav_decoder = tf.audio.decode_wav(wav_loader, desired_channels=1)
        wav_data = sess.run(
        wav_decoder, feed_dict={
            wav_filename_placeholder: wav_dir
        }).audio.flatten()
        sess.close()
    # audio has a sample rate of 16000 and the produced wav has a shape of (16000, 1)
    # reshape to (1, 1600)
    wav = wav_data.reshape(1, 16000)
    wav_new = wav[np.newaxis, :] # introduce a new axis to have a shape of (1, 1, 16000)
    height = wav_new.shape[1]
    width = wav_new.shape[2]
    # create a model to compute spectrogram
    model = tf.keras.Sequential() 
    if compute_type == "spectrogram":
        model.add(Spectrogram(n_dft=512, n_hop=256, input_shape=(height, width),
                            return_decibel_spectrogram=True, power_spectrogram=2.0,
                            trainable_kernel=False, name='static_stft'))
    elif compute_type == "melspectrogram":
        model.add(Melspectrogram(sr=16000, n_mels=128,n_dft=512, n_hop=256,
                            input_shape=(height, width), return_decibel_melgram=True,
                            trainable_kernel=False, name='melgram'))
   
    # producing a spectrogram/melspectrogram from the model
    pred = model.predict(x=wav_new)

    if tf.keras.backend.image_data_format() == "channel_first":
        result = pred[0, 0]
    else:
        result = pred[0, :, :, 0]
    librosa.display.specshow(result, y_axis='linear', sr=16000)


In [ ]:
def get_wav_paths(speaker):
    speaker_path = data_dir + speaker
    all_paths = [item for item in os.listdir(speaker_path)]
    return all_paths

In [ ]:
nelson_mandela_paths = get_wav_paths("/Nelson_Mandela")
margaret_thatcher_paths = get_wav_paths("/Magaret_Tarcher")
benjamin_netanyau_paths = get_wav_paths("/Benjamin_Netanyau")
jens_stoltenberg_paths = get_wav_paths( '/Jens_Stoltenberg')
julia_gillard_paths = get_wav_paths("/Julia_Gillard")

In [ ]:
def load_wav(wav_path, speaker):
    with tf.compat.v1.Session(graph=tf.compat.v1.Graph()) as sess:
        wav_path = data_dir +speaker + "/"+ wav_path
        wav_filename_placeholder = tf.compat.v1.placeholder(tf.compat.v1.string, [])
        wav_loader = tf.io.read_file(wav_filename_placeholder)
        wav_decoder = tf.audio.decode_wav(wav_loader, desired_channels=1)
        wav_data = sess.run(
            wav_decoder, feed_dict={
                wav_filename_placeholder: wav_path
            }).audio.flatten().reshape((1, 16000))
        sess.close()
    return wav_data

In [ ]:
def generate_training_data(speaker_paths, speaker, label):
    wavs, labels = [], []
    for i in tqdm(speaker_paths):
        wav = load_wav(i, speaker)
        wavs.append(wav)
        labels.append(label)
    return wavs, labels

In [ ]:
nelson_mandela_wavs, nelson_mandela_labels = generate_training_data(nelson_mandela_paths, "/Nelson_Mandela", 0) 
margaret_thatcher_wavs, margaret_thatcher_labels = generate_training_data(margaret_thatcher_paths, "/Magaret_Tarcher", 1) 
benjamin_netanyau_wavs, benjamin_netanyau_labels = generate_training_data(benjamin_netanyau_paths, "/Benjamin_Netanyau", 2) 
jens_stoltenberg_wavs, jens_stoltenberg_labels = generate_training_data(jens_stoltenberg_paths, "/Jens_Stoltenberg", 3) 
julia_gillard_wavs, julia_gillard_labels = generate_training_data(julia_gillard_paths, "/Julia_Gillard", 4) 

100%|██████████| 1501/1501 [23:43<00:00,  1.05it/s]


In [ ]:
julia_gillard_labels = julia_gillard_labels[1:]
julia_gillard_wavs = julia_gillard_wavs[1:]

In [ ]:
all_wavs = nelson_mandela_wavs + margaret_thatcher_wavs + benjamin_netanyau_wavs + jens_stoltenberg_wavs + julia_gillard_wavs
all_labels = nelson_mandela_labels + margaret_thatcher_labels + benjamin_netanyau_labels + jens_stoltenberg_labels + julia_gillard_labels

In [ ]:
train_wavs, test_wavs, train_labels, test_labels = train_test_split(all_wavs, all_labels, test_size=0.2)

In [ ]:
train_x, train_y = np.array(train_wavs), np.array(train_labels)
test_x, test_y = np.array(test_wavs), np.array(test_labels)

In [ ]:
train_y = tf.keras.utils.to_categorical(train_y)
test_y = tf.keras.utils.to_categorical(test_y)

In [ ]:
def create_model(speech_feature):
    model = tf.keras.Sequential()
    if speech_feature == "spectrogram":
        model.add(Spectrogram(n_dft=512, n_hop=256, input_shape=(1, 16000),
                            return_decibel_spectrogram=True, power_spectrogram=2.0,
                            trainable_kernel=False, name='static_stft'))
    elif speech_feature == "melspectrogram":
        model.add(Melspectrogram(sr=16000, n_mels=128,n_dft=512, n_hop=256,
                            input_shape=(1 , 16000),return_decibel_melgram=True,
                            trainable_kernel=False, name='melgram'))
   

    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(5, activation="softmax"))
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=3e-4)
            , loss = "categorical_crossentropy"
            , metrics = ["accuracy"])
    return model

In [ ]:
model = create_model("spectrogram")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 static_stft (Spectrogram)   (None, 257, 63, 1)        263168    
                                                                 
 conv2d (Conv2D)             (None, 255, 61, 64)       640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 30, 64)      0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 243840)            0         
                                                                 
 dense (Dense)               (None, 5)                 1219205   
                                                                 
Total params: 1,483,013
Trainable params: 1,483,013
Non-trainable params: 0
______________________________________________

In [ ]:
model.fit(x=train_x, y=train_y, epochs=10, validation_data=(test_x, test_y))

Train on 6000 samples, validate on 1500 samples
Epoch 1/10
6000/6000 [==============================] - ETA: 0s - loss: 27.5081 - accuracy: 0.5957

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


6000/6000 [==============================] - 21s 3ms/sample - loss: 27.5081 - accuracy: 0.5957 - val_loss: 2.0763 - val_accuracy: 0.7587
Epoch 2/10
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.8674 - accuracy: 0.8842 - val_loss: 1.3575 - val_accuracy: 0.8313
Epoch 3/10
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.3117 - accuracy: 0.9495 - val_loss: 0.3273 - val_accuracy: 0.9413
Epoch 4/10
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.5466 - accuracy: 0.9218 - val_loss: 4.7793 - val_accuracy: 0.7033
Epoch 5/10
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.1728 - accuracy: 0.9705 - val_loss: 0.2182 - val_accuracy: 0.9593
Epoch 6/10
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.0707 - accuracy: 0.9845 - val_loss: 0.2925 - val_accuracy: 0.9540
Epoch 7/10
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.0327 - accuracy: 0.9930 - val_loss: 0.2867 - val_accu

In [ ]:
model = create_model("melspectrogram")

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 melgram (Melspectrogram)    (None, 128, 63, 1)        296064    
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 61, 64)       640       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 120960)            0         
                                                                 
 dense_1 (Dense)             (None, 5)                 604805    
                                                                 
Total params: 901,509
Trainable params: 901,509
Non-trainable params: 0
________________________________________________

In [ ]:
model.fit(x=train_x, y=train_y, epochs=10, validation_data=(test_x, test_y))

Train on 6000 samples, validate on 1500 samples
Epoch 1/10
6000/6000 [==============================] - 7s 1ms/sample - loss: 17.8086 - accuracy: 0.6742 - val_loss: 1.6485 - val_accuracy: 0.7907
Epoch 2/10
6000/6000 [==============================] - 7s 1ms/sample - loss: 1.6571 - accuracy: 0.8147 - val_loss: 2.0164 - val_accuracy: 0.8180
Epoch 3/10
6000/6000 [==============================] - 7s 1ms/sample - loss: 1.8625 - accuracy: 0.8425 - val_loss: 1.4218 - val_accuracy: 0.8580
Epoch 4/10
6000/6000 [==============================] - 7s 1ms/sample - loss: 1.8350 - accuracy: 0.8607 - val_loss: 0.6174 - val_accuracy: 0.9267
Epoch 5/10
6000/6000 [==============================] - 7s 1ms/sample - loss: 0.7726 - accuracy: 0.9163 - val_loss: 0.9832 - val_accuracy: 0.8807
Epoch 6/10
6000/6000 [==============================] - 7s 1ms/sample - loss: 0.8860 - accuracy: 0.9068 - val_loss: 1.1682 - val_accuracy: 0.8860
Epoch 7/10
6000/6000 [==============================] - 7s 1ms/sample - los